In [1]:
import numpy as np
import os
import matplotlib.pyplot as plt
import matplotlib
import pandas as pd
import matplotlib.gridspec as gridspec

import replay_structure.structure_models as models
import replay_structure.metadata as meta
import replay_structure.read_write as read_write
import replay_structure.utils as utils

import importlib

import pickle

## Calculate SCALING FACTOR

In [5]:
DATA_PATH = "/home/masha/Documents/Studium/MIT/project/data/"
filename = DATA_PATH+'preprocessed_5cm.obj'
with open(filename, "rb") as file_object:
    raw_data = file_object.read()
    ratday_data = pickle.loads(raw_data)
filename = DATA_PATH+'spikemat_5cm_15ms.obj'
with open(filename, "rb") as file_object:
    raw_data = file_object.read()
    ripple_data = pickle.loads(raw_data)
filename = DATA_PATH+'run_snippets_spikemat_5cm_15ms.obj' 
with open(filename, "rb") as file_object:
    raw_data = file_object.read()
    run_data = pickle.loads(raw_data)

In [12]:
def select_run_periods(ratday, run_period_threshold_s=2):
    run_lengths = (
        ratday.velocity_info["run_ends"] - ratday.velocity_info["run_starts"]
    )
    run_periods_use = run_lengths > run_period_threshold_s
    run_starts = ratday.velocity_info["run_starts"][run_periods_use]
    run_ends = ratday.velocity_info["run_ends"][run_periods_use]
    return np.vstack((run_starts, run_ends)).T

run_times_s = select_run_periods(ratday_data)
run_period_trajectories_cm = utils.get_trajectories(
        ratday_data, run_times_s
    )

In [9]:
def find_best_fit_line(run_mean_frs, ripple_mean_frs):
    fit = np.linalg.lstsq(run_mean_frs[:,None], ripple_mean_frs)
    return fit[0]

def find_best_fit_line_alldata(movement_avg_fr_dict, swr_avg_fr_dict):
    all_movement = np.array([0])
    all_swr = np.array([0])
    for i in range(len(movement_avg_fr_dict)):
        all_movement = np.append(all_movement, movement_avg_fr_dict[i])
        all_swr = np.append(all_swr, swr_avg_fr_dict[i])
    return find_best_fit_line(all_movement, all_swr)

run_mean_frs = ratday_data.place_field_data["mean_firing_rate_array"][ratday_data.place_field_data["place_cell_ids"]]
ripple_mean_frs = ripple_data.ripple_info["popburst_mean_firing_rate_array"]
slope_best_fit = find_best_fit_line(run_mean_frs, ripple_mean_frs)

In [10]:
PF_SCALING_FACTOR = slope_best_fit.mean().round(1)
PF_SCALING_FACTOR

2.5

## Calculate VELOCITY_SCALING_FACTOR

In [ ]:
DATA_PATH = "/home/masha/Documents/Studium/MIT/project/data/"
filename = DATA_PATH+'_5cm_15ms_poisson_trajectories.obj'
with open(filename, "rb") as file_object:
    raw_data = file_object.read()
    ripple_trajectories = pickle.loads(raw_data)

In [13]:
def get_total_spikes_vec(spikemats):
    n_traj = len(spikemats)
    total_spikes_vec = np.full(n_traj, np.nan)
    for i in range(n_traj):
        if spikemats[i] is not None:
            total_spikes_vec[i] = spikemats[i].sum()
    return total_spikes_vec

def get_total_distance(trajectory):
    return np.sum(np.sqrt(np.sum((trajectory[1:] - trajectory[:-1])**2, axis=1)))

def get_velocity(trajectory, time_window_s):
    distance = get_total_distance(trajectory)
    duration_s = len(trajectory) * time_window_s
    velocity = distance/duration_s
    return velocity

def get_velocity_vec(trajectories, time_window_s):
    n_traj = len(trajectories)
    velocity_cm_per_s = np.full(n_traj, np.nan)
    for i in range(n_traj):
        if trajectories[i] is not None:
            velocity_cm_per_s[i] = get_velocity(trajectories[i], time_window_s)
    return velocity_cm_per_s

def get_distance_vec(trajectories):
    n_traj = len(trajectories)
    distance_vec = np.full(n_traj, np.nan)
    for i in range(n_traj):
        if trajectories[i] is not None:
            distance_vec[i] = get_total_distance(trajectories[i])
    return distance_vec[~np.isnan(distance_vec)]

def get_n_time_windows(spikemats):
    n_traj = len(spikemats)
    total_spikes_vec = np.full(n_traj, np.nan)
    for i in range(n_traj):
        if spikemats[i] is not None:
            total_spikes_vec[i] = spikemats[i].shape[0]
    return total_spikes_vec[~np.isnan(total_spikes_vec)]

In [ ]:
velocity_ripples = get_velocity_vec(ripple_trajectories.most_likely_trajectories, .003)
velocity_run_periods = get_velocity_vec(run_period_trajectories_cm[str(session)], 1/30)
mean_velocity_ripples[i] = np.nanmean(velocity_ripples[str(session)])
mean_velocity_run_periods[i] = np.nanmean(velocity_run_periods[str(session)])
sd_velocity_ripples[i] = np.nanstd(velocity_ripples[str(session)])/np.sqrt(meta.N_SESSIONS)
sd_velocity_run_periods[i] = np.nanstd(velocity_run_periods[str(session)])/np.sqrt(meta.N_SESSIONS)

In [ ]:
VELOCITY_SCALING_FACTOR = np.mean(mean_velocity_ripples/mean_velocity_run_periods).round(1)

## Calculate MAX_LIKELIHOOD_SD_METERS_RIPPLES and MAX_LIKELIHOOD_SD_METERS_RUN_SNIPPETS

In [ ]:
# load gridsearch result for ripple data
bin_size_cm = 5
time_window_ms = 15
RESULT_PATH = "/home/masha/Documents/Studium/MIT/project/1D/results"
filename = RESULT_PATH+'/ripples_'+str(bin_size_cm)+'cm_'+str(time_window_ms)+'ms_poisson.obj'
with open(filename, "rb") as file_object:
    raw_data = file_object.read()
    dif_marginalized_gridsearch = pickle.loads(raw_data)

In [ ]:
dif_sd_meters = dif_marginalized_gridsearch.marginalization_info["best_fit_gridsearch_params"]["sd_meters"]

In [ ]:
# mode from diffusion gridsearch on all sessions
MAX_LIKELIHOOD_SD_METERS_RIPPLES = stats.mode(dif_sd_meters)
MAX_LIKELIHOOD_SD_METERS_RIPPLES